# Homework 2 - Wikipedia Web Traffic Time Series

У вас є дані про відвідування 1000 сторінок Вікіпедії з різних країн та різних девайсів (дані взяті з [Kaggle змагання](https://www.kaggle.com/c/web-traffic-time-series-forecasting))

*wikipedia_train* и *wikipedia_test* - містять дані про трафік. Це файли CSV, де кожен рядок відповідає певній статті, і кожен стовпець відповідає конкретній даті. У деяких записах дані відсутні. Назви сторінок містять проект Вікіпедії (наприклад, en.wikipedia.org), тип доступу (наприклад, desktop) та тип агента (наприклад, spider). Кожне ім'я статті має такий формат: «name_project_access_agent» (наприклад, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам треба відповісти на [питання](https://forms.gle/tx4gSVpCn6cvHMYY6) і спробувати зробити найпростішу модель, яка зможе передбачати майбутні відвідування.

Ось приклади часових рядів відвідуваності сторінок Вікіпедії (*сині* - навчальна вибірка, *зелені* - передбачення моделі переможця змагання на Kaggle, *помаранчеві* - реальні значення):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [ ]:
import re
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv("../data/wikipedia_train.csv")
test = pd.read_csv("../data/wikipedia_test.csv")

## Data Analysis

In [ ]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

## Forecasting

Потрібно перетворитиь `train` дані в наступний формат:

In [ ]:
train.head()

,Page,date,Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24


Таким чином у вас кожен рядок містить набір фіч (`Page`, `date`) та цільову змінну (`Visits`). Перетворити дані на такий формат допоможе функція `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

Оцінювати якість передбачень ми будемо за допомогою [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [ ]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

Щоб оцінити точність прогнозу потрібно перетворити `test` дані в наступний формат:

In [ ]:
prediction.head()

,Page,date,Visits,pred_Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2016-09-10,43,37
1,2012_(film)_fr.wikipedia.org_all-access_spider,2016-09-10,6,5
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2016-09-10,4,4
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2016-09-10,15,14
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2016-09-10,101,173


І потім викликати функцію для підрахунку метрики:

In [ ]:
pandas_smape(prediction)

### Last day baseline

Потрібно зробити прогноз на основі відвідувань в останній відомий нам день з train (продублювати значення для кожного дня у test)

### Median baseline

Потрібно зробити прогноз на основі медіани за останні **30** днів з `train`. 

А потім покращити передбачення використовуючи інформацію вихідний це чи ні (скористайтеся функцією [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) та різні вікна для підрахунку медіани (7 днів, 60 днів тощо)

Вам допоможе функція `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)